In [44]:
#https://www.kaggle.com/c/global-energy-forecasting-competition-2012-load-forecasting

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral4, Spectral6, Spectral8, Spectral11 #color palette
import calendar

output_notebook()

colorRange =  ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928']

%matplotlib inline

Loading BokehJS ...

In [45]:
load = pd.read_csv("Load_history.csv", thousands=",")
temperature = pd.read_csv("temperature_history.csv")

In [46]:
load.head()

,zone_id,year,month,day,h1,h2,h3,h4,h5,h6,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
0,1,2004,1,1,16853,16450,16517,16873,17064,17727,...,13518,13138,14130,16809,18150,18235,17925,16904,16162,14750
1,1,2004,1,2,14155,14038,14019,14489,14920,16072,...,16127,15448,15839,17727,18895,18650,18443,17580,16467,15258
2,1,2004,1,3,14439,14272,14109,14081,14775,15491,...,13507,13414,13826,15825,16996,16394,15406,14278,13315,12424
3,1,2004,1,4,11273,10415,9943,9859,9881,10248,...,14207,13614,14162,16237,17430,17218,16633,15238,13580,11727
4,1,2004,1,5,10750,10321,10107,10065,10419,12101,...,13845,14350,15501,17307,18786,19089,19192,18416,17006,16018


## We manipulate the dataset to have a column based dataframe

In [47]:
columns = [x for x in load.columns if x.startswith("h")] #retrieve the columns with data

In [48]:
load["date"]=load["day"].map(str) + "/"+ load["month"].map(str) + "/" + load["year"].map(str)
load["date"] = pd.to_datetime(load["date"], format="%d/%m/%Y")
load.set_index("date", inplace=True)
data= load.loc[load["zone_id"]==1, columns].stack()
data= data.reset_index()
data.fillna(method="pad", inplace=True, limit=1)

In [49]:
data["hour"] = (data.index.values) % 24
data["date"]= pd.to_datetime(data["date"]+pd.to_timedelta(data["hour"], unit='h')) #Add hour to the datetime
data.drop(["level_1", "hour"], axis=1, inplace=True)
data.set_index("date", inplace=True)
data.columns=["Zone_id 1"]

In [50]:
#Similar process for the temperature data
temperature["date"] = pd.to_datetime(temperature["day"].map(str) + "/" 
                                     + temperature["month"].map(str) + "/" 
                                     + temperature["year"].map(str), format="%d/%m/%Y")
temperature.set_index("date", inplace=True)

frames=[]
for station in temperature["station_id"].unique():
    frames.append(temperature.loc[temperature["station_id"]==station,columns].stack())

tempData = pd.concat(frames,axis=1)
tempData.columns= ["Station_id " + str(x+1) for x in tempData.columns]

In [51]:
tempData["hour"] = [int(x.split("h")[1]) for x in tempData.index.get_level_values(1)]
tempData["date"] = tempData.index.get_level_values(0)
tempData["date"] = pd.to_datetime(tempData["date"] + pd.to_timedelta(tempData["hour"], unit='h'))
tempData.set_index("date", inplace=True)

In [52]:
#Reindex the dataset adding the missing period that need to be backasted
from datetime import datetime
start_date = datetime(2004, 1,1, 1) # start date 01/01/2004
end_date = datetime(2008,7,8) # end date 08/07/2008
rng=pd.date_range(start=start_date, end=end_date , freq='"H')
data.set_index(pd.to_datetime(data.index), inplace=True)
result = data.reindex(rng)
result.head()

,Zone_id 1
2004-01-01 01:00:00,16450
2004-01-01 02:00:00,16517
2004-01-01 03:00:00,16873
2004-01-01 04:00:00,17064
2004-01-01 05:00:00,17727


## Join the consumption and the temperature data

In [53]:
zone1Data = pd.concat([result,tempData], axis=1, join="outer")
zone1Data["day"] = zone1Data.index.dayofyear
zone1Data["hour"] = zone1Data.index.hour
zone1Data["weekday"] = zone1Data.index.dayofweek
zone1Data["month"] =zone1Data.index.month
zone1Data["year"]=zone1Data.index.year
zone1Data["week"]=zone1Data.index.week
# zone1Data.to_csv("zone1.csv") #save the file for future use

# Quick analysis of the consumption pattern of the zone 1

In [54]:
#Group the data in different ways
dailyData = zone1Data.groupby(["year", "day"], axis=0).sum().reset_index() #group per year and day
weeklyData = zone1Data.groupby(["year", "week"], axis=0).sum().reset_index() #group per year and week 
weekbyHourData = zone1Data.groupby(["year", "weekday", "hour"], axis=0).mean().reset_index() #group per year, week and hour

## Based on daily data, there is seasonality in the consumption. There is a unexpected drop of consumption between 350 and 400 probably due to some outages. 

In [55]:
plot2 = figure(width=1000, height=600)
plot2.background_fill_color = "whitesmoke"
plot2.xaxis.axis_label="Day"
plot2.yaxis.axis_label="Energy Consumption"
xaxis = dailyData.index.values
plot2.line(xaxis, dailyData["Zone_id 1"], line_width=2, color=colorRange[1])
show(plot2)

## We rediscover the seasonality on the weekly graph. Higher consumption during winter and summer time. As well as some outages...

In [56]:
plot3 = figure(width=1000, height=600)
plot3.background_fill_color = "whitesmoke"
plot3.xaxis.axis_label="Week number"
plot3.yaxis.axis_label="Energy Consumption"
xaxis = weeklyData.index.values

for index, year in enumerate(weeklyData["year"].unique()):
    plot3.line(xaxis, weeklyData.loc[weeklyData["year"]==year, ["Zone_id 1"]], line_width=2, color=colorRange[index], legend=str(year))
    
show(plot3)

## Until 2008, all days of the week follow a similar pattern. 2008 shows some disparities but there is data for only half of the year.

In [57]:
plot3 = figure(width=1000, height=600)
plot3.background_fill_color = "whitesmoke"
plot3.xaxis.axis_label="Week by hour"
plot3.yaxis.axis_label="Energy Consumption"
xaxis = weekbyHourData.index.values

for index, year in enumerate(weekbyHourData["year"].unique()):
    plot3.line(xaxis, weekbyHourData.loc[weekbyHourData["year"]==year, ["Station_id 1"]], line_width=2, color=colorRange[index], legend=str(year))

show(plot3)

## There is a strong correlation between consumption and outdoor temperature. Electricity is likely to be used for both heating and cooling.

In [58]:
dailyDataAvg = zone1Data.groupby(["year", "day"], axis=0).mean().reset_index()

plot3 = figure(width=1000, height=600)
plot3.background_fill_color = "whitesmoke"
plot3.xaxis.axis_label="Temperature in F"
plot3.yaxis.axis_label="Avg consumption"
plot3.circle(dailyDataAvg["Station_id 1"],dailyDataAvg["Zone_id 1"] , size=5, color="navy", alpha=0.5)
show(plot3)